In [5]:
import pandas as pd

In [6]:
# Load cleaned adjusted close prices
df = pd.read_csv('../data/sp100_selected_data.csv', index_col=0, parse_dates=True)

# Ensure all values are numeric
df = df.apply(pd.to_numeric, errors='coerce')
df = df.fillna(method='ffill').fillna(method='bfill')


/var/folders/tt/1wm0x7wj6sg2zfqbrjhzrtrc0000gn/T/ipykernel_7627/3011863545.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('../data/sp100_selected_data.csv', index_col=0, parse_dates=True)
/var/folders/tt/1wm0x7wj6sg2zfqbrjhzrtrc0000gn/T/ipykernel_7627/3011863545.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')


In [7]:
# Calculate daily returns
daily_returns = df.pct_change().dropna()

# Calculate cumulative returns
cumulative_returns = (1 + daily_returns).cumprod()


In [8]:
# Define q = number of stocks to select
q = 10

# Drop ^OEX (benchmark) before sorting
correlations = cumulative_returns.corr()['^OEX'].drop('^OEX')

# Select top q stocks most correlated with the S&P 100
top_q_stocks = correlations.sort_values(ascending=False).head(q).index.tolist()

# Display selected stock tickers
top_q_stocks


['^OEX.1',
 '^OEX.2',
 '^OEX.3',
 'AAPL.2',
 'AAPL',
 'AAPL.3',
 'AAPL.1',
 'JPM.2',
 'JPM',
 'JPM.1']

In [9]:
# Filter the daily returns for selected stocks and benchmark
returns_for_ampl = daily_returns[top_q_stocks + ['^OEX']]

# Show the shape and head of the data
returns_for_ampl.shape, returns_for_ampl.head()


((252, 11),
               ^OEX.1    ^OEX.2    ^OEX.3    AAPL.2      AAPL    AAPL.3  \
 Ticker                                                                   
 Date        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
 2024-04-01  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
 2024-04-02 -0.010150 -0.007943 -0.006588 -0.007376 -0.012326 -0.006999   
 2024-04-03  0.003801  0.002614  0.000581  0.002081 -0.001715  0.004797   
 2024-04-04  0.005235 -0.009843 -0.012935  0.001424  0.008887 -0.004892   
 
               AAPL.1     JPM.2       JPM     JPM.1      ^OEX  
 Ticker                                                        
 Date        0.000000  0.000000  0.000000  0.000000  0.000000  
 2024-04-01  0.000000  0.000000  0.000000  0.000000  0.000000  
 2024-04-02 -0.011153 -0.002468 -0.008000 -0.005773 -0.010852  
 2024-04-03  0.007913 -0.001918  0.003276 -0.001101  0.000236  
 2024-04-04  0.007265 -0.005105  0.006389  0.006438  0.009844  )

In [10]:
# Average daily return for each stock (can be scaled later)
avg_returns = returns_for_ampl.mean()

# Print average return of each selected stock and the benchmark
avg_returns


^OEX.1    0.000377
^OEX.2    0.000317
^OEX.3    0.000403
AAPL.2    0.001113
AAPL      0.001104
AAPL.3    0.001199
AAPL.1    0.001223
JPM.2     0.000900
JPM       0.000910
JPM.1     0.000990
^OEX      0.000334
dtype: float64

In [11]:
# Format set of stocks
stock_set = "set STOCKS := " + ' '.join(top_q_stocks) + ";\n"

# Format return parameters
return_param = "param r :=\n"
for stock in top_q_stocks:
    return_param += f"    {stock} {avg_returns[stock]:.6f}\n"
return_param += ";\n"

# Combine into .dat content
ampl_dat_content = stock_set + "\n" + return_param

# Save to file
with open('../optimization/index_data.dat', 'w') as f:
    f.write(ampl_dat_content)

# Print preview
print(ampl_dat_content)


set STOCKS := ^OEX.1 ^OEX.2 ^OEX.3 AAPL.2 AAPL AAPL.3 AAPL.1 JPM.2 JPM JPM.1;

param r :=
    ^OEX.1 0.000377
    ^OEX.2 0.000317
    ^OEX.3 0.000403
    AAPL.2 0.001113
    AAPL 0.001104
    AAPL.3 0.001199
    AAPL.1 0.001223
    JPM.2 0.000900
    JPM 0.000910
    JPM.1 0.000990
;



In [12]:
print(f"target_return: {avg_returns['^OEX']:.6f}")

target_return: 0.000334
